# Лекция 1 
# Базовый SQL 

In [ ]:
!pip install ipython-sql #не обязательно запускать в colab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
%load_ext sql
%sql sqlite://

'Connected: @None'

In [3]:
%%sql 
select 'Hello, world';

 * sqlite://
Done.


"'Hello, world'"
"Hello, world"


Пара замечаний:
 * Мы используем веб-интерфейс для python, поэтому для запуска SQL запросов необходимо применить ряд вещей:
     * Необходимо подключить расширение SQL через так называемые magic command. [Более подробно здесь](https://ipython.readthedocs.io/en/stable/interactive/magics.html)
     * Для работы с SQL надо вызвать либо %sql для однострочной команды, либо %%sql - для многострочной
     * При использовании выражения SELECT результатом вывода является таблица, но при этом в notebook выводится уже внутренне представление языка python (объект класса sql.run.resultset, подробности далее), что приводит к ряду несоответствий (например, None вместо NULL)

Давайте создадим таблицу, заполним ее и сделаем какой-нибудь запрос!

In [4]:
%%sql

create table product(
       pname        varchar(30) primary key,  -- имя продукта
       price        money,                    -- цена продукта
       category     varchar(15),              -- категория
       manufacturer varchar(30) NOT NULL      -- производитель
);

insert into product (pname, price, category, manufacturer) values 
('Тетрадь', 39.99, 'Канцелярия', 'Академия холдинг');

insert into product values('Клавиатура', 949.99, 'Техника', 'Sven');

insert into product 
    values ('Степлер', 129.99, 'Канцелярия', 'Brauberg'),
           ('Батарейка', 39.99, 'Для дома', 'Energizer'),
           ('Энергетик', 89.00, 'Напитки', 'Redbull');

 * sqlite://
Done.
1 rows affected.
1 rows affected.
3 rows affected.


[]

In [5]:
%%sql
--insert into product (pname, category) values ('Маркер', 'Канцелярия');

 * sqlite://
0 rows affected.


[]

Посмотрим на полученную таблицу.

In [6]:
%sql select * from product;

 * sqlite://
Done.


pname,price,category,manufacturer
Тетрадь,39.99,Канцелярия,Академия холдинг
Клавиатура,949.99,Техника,Sven
Степлер,129.99,Канцелярия,Brauberg
Батарейка,39.99,Для дома,Energizer
Энергетик,89,Напитки,Redbull


# Немного SQL терминологии
* _имя_ таблицы - product.
* Каждая строка таблицы называется _строкой_ или _кортеж_. 
* Заметьте, что все кортежи имеют _поля_ или _атрибуты_.
* Количество строк называет _мощностью_, в то время как количество атрибутов _арностью_ 

# Простые запросы
* Рассмотрим основы SQL на примере
* В интернете очень много SQL туториалов, сейчас мы всего лишь вспомним базовые вещи

> SELECT <Множество атрибутов><br>
> FROM <список таблиц и условие на соединение><br>
> WHERE <список условий>

Это простейший SELECT-FROM-WHERE (SFW) блок. Давайте посмотрим на примерах!

In [7]:
%%sql
SELECT * 
FROM Product 
WHERE category = 'Канцелярия' AND manufacturer = 'Brauberg';

 * sqlite://
Done.


pname,price,category,manufacturer
Степлер,129.99,Канцелярия,Brauberg


\* \- В данном случае обозначает вывод всех полей, которые были описаны во FROM

Посмотрим на примеры *проекции*, то есть получим только несколько атрибутов запроса.

In [8]:
%%sql 
SELECT Pname, 
       Price, 
       Manufacturer
  FROM Product;

 * sqlite://
Done.


pname,price,manufacturer
Тетрадь,39.99,Академия холдинг
Клавиатура,949.99,Sven
Степлер,129.99,Brauberg
Батарейка,39.99,Energizer
Энергетик,89,Redbull


* На выходе *все еще* таблица и ее схема -
> Answer(pname, price, manufacturer)

* Можно объединять выборку и проекцию + менять результат

In [9]:
%%sql
SELECT Pname, Price * 10 as price_x2, Manufacturer
FROM Product
WHERE category in ('Напитки', 'Канцелярия');

 * sqlite://
Done.


pname,price_x2,manufacturer
Тетрадь,399.90000000000003,Академия холдинг
Степлер,1299.9,Brauberg
Энергетик,890,Redbull


In [10]:
%%sql
SELECT pname || '->' || price || '->' || manufacturer as concat
FROM Product
WHERE category = 'Напитки';

 * sqlite://
Done.


concat
Энергетик->89->Redbull


На выходе запроса к таблице - снова таблица 

In [11]:
%%sql
SELECT manufacturer
FROM Product p0
WHERE p0.price < 100.00;

 * sqlite://
Done.


manufacturer
Академия холдинг
Energizer
Redbull


In [12]:
%%sql
SELECT
    p.manufacturer, p.pname, p.price
FROM 
    (SELECT p0.manufacturer
     FROM Product p0
     WHERE p0.price < 100.00) cp, -- вложенный запрос
    Product p
WHERE 
    p.manufacturer = cp.manufacturer AND p.category = 'Канцелярия';

 * sqlite://
Done.


manufacturer,pname,price
Академия холдинг,Тетрадь,39.99


Небольшие детали
--------------

* Некоторые элементы регистро-независимые:
  * Одно и то же: SELECT  Select  select
  * Одно и то же: Product   product
  * Разные: ‘Brauberg’  ‘brauberg’ (Здесь это строковая константа)



In [13]:
%%sql 
select * from product;

 * sqlite://
Done.


pname,price,category,manufacturer
Тетрадь,39.99,Канцелярия,Академия холдинг
Клавиатура,949.99,Техника,Sven
Степлер,129.99,Канцелярия,Brauberg
Батарейка,39.99,Для дома,Energizer
Энергетик,89,Напитки,Redbull


LIKE
====
Регулярные выражения

Опертор LIKE нужен для поиска строк:
    
> SELECT *
> FROM Products
> WHERE pname like '%подстрока%'

* % - сколько угодно символов
* \_ ровно один символ
* оператор LIKE - регистрозависимый


In [14]:
%%sql
SELECT * FROM product
where category LIKE '%ка%';

 * sqlite://
Done.


pname,price,category,manufacturer
Клавиатура,949.99,Техника,Sven


In [15]:
%%sql
SELECT * FROM product
where category LIKE '_а%';

 * sqlite://
Done.


pname,price,category,manufacturer
Тетрадь,39.99,Канцелярия,Академия холдинг
Степлер,129.99,Канцелярия,Brauberg
Энергетик,89,Напитки,Redbull


Убрать дубли
---------------------
* Дубли не всегда хорошо, и иногда их стоит убирать
  * Помните, что таблицы - _мультимножества_!

In [16]:
%%sql 
SELECT category from product;

 * sqlite://
Done.


category
Канцелярия
Техника
Канцелярия
Для дома
Напитки


In [17]:
%%sql 
-- чтобы убрать дубли используйте слово DISTINCT
SELECT DISTINCT category from product;

 * sqlite://
Done.


category
Канцелярия
Техника
Для дома
Напитки


Сортировка результатов
---------------------
* Так как Таблица - это мультимножество, то порядок вывода строк не гарантирован. Иногда необходимо выводить строки в определенном порядке


In [18]:
%%sql
-- сортировка результатов
-- сортировка по умолчанию - ascending
SELECT   pname, price, manufacturer
FROM     Product
WHERE    price < 400
ORDER BY price, manufacturer;

 * sqlite://
Done.


pname,price,manufacturer
Батарейка,39.99,Energizer
Тетрадь,39.99,Академия холдинг
Энергетик,89,Redbull
Степлер,129.99,Brauberg


In [19]:
%%sql
-- сортировка результатов
-- тип сортировки каждого компонента определяется индивидуально
SELECT   price, manufacturer
FROM     Product
ORDER BY   manufacturer ASC, price DESC;

 * sqlite://
Done.


price,manufacturer
129.99,Brauberg
39.99,Energizer
89,Redbull
949.99,Sven
39.99,Академия холдинг


Можно делать сортировку по порядковому номеру, но довольно часто это считается bad practice 

In [20]:
%%sql
SELECT   price, manufacturer
FROM     Product
ORDER BY 2 ASC, 1 DESC;

 * sqlite://
Done.


price,manufacturer
129.99,Brauberg
39.99,Energizer
89,Redbull
949.99,Sven
39.99,Академия холдинг


Работа с несколькими таблицами
------


* Рассмотрим таблицу компаний.
> company(<u>cname</u>, production, country)
 

In [21]:
%%sql
drop table if exists product;
drop table if exists company;

create table company (
    cname varchar primary key,    -- Имя
    production varchar,           -- Продукция
    country varchar);             -- Страна

insert into company values ('Brauberg', 'Товары для офиса', 'Россия');
insert into company values ('Energizer', 'Аккумуляторы', 'США');
insert into company values ('Sven', 'Акустическая система и компьютерная периферия', 'Финляндия');

 * sqlite://
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.


[]

Внешние ключи
-----------------------
* Допустим мы ходим добавить таблицу продуктов

> Product(pname, price, category, manufacturer)

* Может возникнуть следующая ситуация: есть компания, продающая какие-то продукты, но при этом она отсутствует в таблице компаний.
* Чтобы избежать это, воспользуемя _внешними ключами_ 

Назвнание компании в product _ссылается_ на поле cname из таблицы company:

> foreign key (manufacturer) references company(cname)

  * Замечание: cname <u>должно быть</u> ключом в  company! 
  

In [22]:
%%sql 

drop table if exists product;

pragma foreign_keys = ON; -- WARNING by default off in sqlite

create table product(
       pname        varchar primary key,  -- имя продукта
       price        money,                -- цена продукта
       category     varchar,              -- категория
       manufacturer varchar NOT NULL,      -- производитель
       foreign key (manufacturer) references company(cname)
);

insert into product values('Клавиатура', 949.99, 'Техника', 'Sven');
insert into product values('Наушники', 1999.99, 'Техника', 'Sven');
insert into product values('Степлер', 129.99, 'Канцелярия', 'Brauberg');
insert into product values('Маркер', 59.99, 'Канцелярия', 'Brauberg');
insert into product values('Батарейка', 39.99, 'Для дома', 'Energizer');

 * sqlite://
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

Внешние ключи являются _ограничениями_ таблицы 
> Что случится при вставке компании, которой не в таблице company?


In [ ]:
%%sql 
--insert into product values('Тетрадь', 39.99, 'Канцелярия', 'Академия холдинг');

In [24]:
%%sql
select * from product;

 * sqlite://
Done.


pname,price,category,manufacturer
Клавиатура,949.99,Техника,Sven
Наушники,1999.99,Техника,Sven
Степлер,129.99,Канцелярия,Brauberg
Маркер,59.99,Канцелярия,Brauberg
Батарейка,39.99,Для дома,Energizer


In [25]:
%%sql 
select * from company;

 * sqlite://
Done.


cname,production,country
Brauberg,Товары для офиса,Россия
Energizer,Аккумуляторы,США
Sven,Акустическая система и компьютерная периферия,Финляндия


Внешние ключи и удаление
=============

* Что произойдет, если мы удалим компанию ? Несколько вариантов:
  * Запретить удаление (default)
  * Удалить все продукты (добавьте "`on delete cascade`")
  * Замена на  NULL или DEFAULT
  

**Первый вариант (default)- Удаление запрещено**

In [ ]:
%sql delete from company where cname = 'Brauberg';

**Второй вариант: удалить все продукты, принадлежащие компании, которую мы удаляем**

Необходимо изменить опеределение в create table:
> foreign key (manufacturer) references company(cname) on delete cascade

Теперь, когда строка компании удалена, все продукты с указанной компанией, будут также удалены.

Соединения
------
> Product (<u>pname</u>,  price, category, manufacturer)<br>
> Company (<u>cname</u>, production, country)

Мы хотим ответить на вопрос

> Найти все продукты меньшие 100р, произведенные в России

> Возвратить их имена и цену. 



In [28]:
%%sql
SELECT pname, price
FROM product, company
where manufacturer=cname and country='Россия' and price < 100;

 * sqlite://
Done.


pname,price
Маркер,59.99


Другой вариант написания запроса.

In [29]:
%%sql 
-- Часть 1: выбрать все компании из России
SELECT distinct cname -- нужна ли нам уникальность?
from company where country='Россия';

 * sqlite://
Done.


cname
Brauberg


In [30]:
%%sql 
-- Часть 2: Продукты до 100
select pname, price, manufacturer
from product
where price <= 100;

 * sqlite://
Done.


pname,price,manufacturer
Маркер,59.99,Brauberg
Батарейка,39.99,Energizer


In [31]:
%%sql 
-- Объединение как cross join
SELECT * 
FROM 
  (SELECT DISTINCT pname, price, manufacturer
   FROM product
   WHERE price <= 100) CheapProducts,
  (SELECT DISTINCT cname
   FROM company
   WHERE country='Россия') RusProducts;

 * sqlite://
Done.


pname,price,manufacturer,cname
Батарейка,39.99,Energizer,Brauberg
Маркер,59.99,Brauberg,Brauberg


In [32]:
%%sql
-- Фильтруем cross join
SELECT DISTINCT pname, price
FROM 
  (SELECT DISTINCT pname, price, manufacturer
   FROM product
   WHERE price <= 100) CheapProducts,
  (SELECT distinct cname
   FROM company
   WHERE country='Россия') RusProducts
WHERE cname = manufacturer;

 * sqlite://
Done.


pname,price
Маркер,59.99


# Использование конструкции JOIN

Есть другой вариант записи - через конструкцию JOIN. Кроме этого она позволяет делать также внешние запросы.



In [33]:
%%sql
SELECT *
  FROM product p 
  JOIN company c 
    ON p.manufacturer = c.cname
 WHERE c.country='Россия' 
   AND p.price <= 100;

 * sqlite://
Done.


pname,price,category,manufacturer,cname,production,country
Маркер,59.99,Канцелярия,Brauberg,Brauberg,Товары для офиса,Россия


Примечания
--------
* Есть множество логических вариантов написать один и тот же запрос
    * Этот факт будет использоваться для оптимизации

Дубли после соединения
--------------------------

Замечание - могут возникнуть дубли после соединения...

In [34]:
%%sql
SELECT Country
FROM Product, Company
WHERE manufacturer=cname AND category='Канцелярия';

 * sqlite://
Done.


country
Россия
Россия


# Группировка 


Можно осуществлять схлопывание строк в рамках каких-нибудь групп

После схлопывания можно применять так называемые агрегатные функции

AVG(<поле>) 

SUM(<поле>)

MIN(<поле>)

MAX(<поле>)

COUNT(<поле>)

COUNT(*)

AVG(distinct <поле>)

SUM(distinct <поле>)

COUNT(distinct <поле>)

In [35]:
%%sql 
select category, avg(price), min(price), max(price), sum(price)
from Product
group by category;

 * sqlite://
Done.


category,avg(price),min(price),max(price),sum(price)
Для дома,39.99,39.99,39.99,39.99
Канцелярия,94.99000000000001,59.99,129.99,189.98000000000002
Техника,1474.99,949.99,1999.99,2949.98


Существует возможность фильтровать не только по строкам, но и по группам. Для этого можно использовать инструкцию HAVING

In [36]:
%%sql 
select category, avg(price), min(price), max(price), count(price)
from Product
group by category
having min(price) < 100;

 * sqlite://
Done.


category,avg(price),min(price),max(price),count(price)
Для дома,39.99,39.99,39.99,1
Канцелярия,94.99000000000001,59.99,129.99,2


In [37]:
%%sql 
select category, count(*) as star, count(category) as cat, count(distinct category) as dist
from Product
group by category;

 * sqlite://
Done.


category,star,cat,dist
Для дома,1,1,1
Канцелярия,2,2,1
Техника,2,2,1


count(*) - кол-во строк

count(<поле>) - кол-во непустых строк

count(distinct <поле>) - кол-во уникальных непустых строк

In [38]:
%%sql 
drop table if exists numbers;

create table numbers(a int, b int);

insert into numbers 
  values (1,2), (2,3), (4,null), (null,null), (null,5);

 * sqlite://
Done.
Done.
5 rows affected.


[]

In [39]:
%%sql 
select * from numbers;

 * sqlite://
Done.


a,b
1,2
2,3
4,None
None,None
None,5


In [40]:
%%sql 
select sum(a+b) as sum1,
       sum(a) + sum(b) as sum2 
       from numbers;

 * sqlite://
Done.


sum1,sum2
8,17


In [41]:
%%sql 
select a || '+' || b as concat
       from numbers;

 * sqlite://
Done.


concat
1+2
2+3
None
None
None


In [42]:
%%sql 
select *
  from numbers
where a>0;

 * sqlite://
Done.


a,b
1,2
2,3
4,None
